Dependencias

In [ ]:
# pip install pandas
# pip install sqlalchemy
# pip install openpyxl

Importaciones

In [5]:
from datetime import datetime
import pandas as pd
from sqlalchemy import create_engine

Importacion de Archivo Fuente

In [8]:
#Import .csv file
df = pd.read_excel('./PADRON+CATALOGO  GENERAL JULIO 2024 CON SUSTITUCIONES.xlsx')
# Guardar el DataFrame como un archivo pickle
df.to_pickle('padron.pkl')
# Leer archivo
df = pd.read_pickle('padron.pkl')

Datos estaticos

In [9]:
# Agregar las columnas "fecha_captura" y "fecha_actualizacion" con valores estáticos de tipo datetime
fecha_captura = datetime(2024, 9, 1)  # Fecha válida de este año y mes (ejemplo: 1 de septiembre de 2024)
fecha_actualizacion = datetime(2024, 9, 1)  # Fecha válida de este año y mes (ejemplo: 2 de septiembre de 2024)

df['fecha_captura'] = fecha_captura
df['fecha_actualizacion'] = fecha_actualizacion
# Agregar la columna "estatus" con valor 1 (entero)
df['estatus'] = 1

Limpieza de Datos

In [10]:
# Convertir la columna 'fecha' a formato datetime, forzando valores inválidos a NaT
df['fecha_entrada'] = pd.to_datetime(df['fecha_entrada'], errors='coerce')

# Reemplazar los caracteres no deseados en todas las columnas de tipo string (object)
for column in df.columns:
    if df[column].dtype == 'object':
        df[column] = df[column].str.replace('_x000D_', '', regex=False).str.replace('_x0002_', '', regex=False).replace('S/N', None, regex=False).replace('S/N-', None, regex=False).replace('SIN NUMERO', None, regex=False).str.strip()

# Filtrar filas donde 'columna1' y 'columna3' tienen valores vacíos
filas_vacias = df[df[['fuente_financiamiento_id', 'numero_economico', 'numero_inventario', 'nombre_completo', 'numero_resguardo']].isnull().all(axis=1)]
# print(filas_vacias)

df_sin_vacios = df.drop(filas_vacias.index)

In [ ]:
print(df_sin_vacios.head())

Insercion

In [ ]:
#INPUT YOUR OWN CONNECTION STRING HERE
conn_string = "postgresql://desarrollo:salud2024@db:5432/departamento_inventario"
#perform to_sql test and print result
db = create_engine(conn_string)
conn = db.connect()

df_sin_vacios.to_sql('BienFisico', con=conn, if_exists='append', index=False, chunksize=1000)